# Assignment 10

It happens all the time: someone gives you data containing malformed strings, Python, lists and missing data. How do you tidy it up so you can get on with the analysis?
Take this monstrosity as the DataFrame to use in the following puzzles:
df = pd.DataFrame({'From_To': ['LoNDon_paris', 'MAdrid_miLAN', 'londON_StockhOlm',
'Budapest_PaRis', 'Brussels_londOn'],
'FlightNumber': [10045, np.nan, 10065, np.nan, 10085],'RecentDelays': [[23, 47], [], [24, 43, 87], [13], [67, 32]],
'Airline': ['KLM(!)', '<Air France> (12)', '(British Airways. )',
'12. Air France', '"Swiss Air"']})

1. Some values in the the FlightNumber column are missing. These numbers are meant to increase by 10 with each row so 10055 and 10075 need to be put in place. Fill in these missing numbers and make the column an integer column (instead of a float column).

In [56]:
import pandas as pd
import numpy as np
df = pd.DataFrame({'From_To': ['LoNDon_paris', 'MAdrid_miLAN', 'londON_StockhOlm', 'Budapest_PaRis', 'Brussels_londOn'], 'FlightNumber': [10045, np.nan, 10065, np.nan, 10085],'RecentDelays': [[23, 47], [], [24, 43, 87], [13], [67, 32]], 'Airline': ['KLM(!)', ' (12)', '(British Airways. )', '12. Air France', '"Swiss Air"']})
df['FlightNumber'] = df['FlightNumber'].interpolate().astype(int)
df

,From_To,FlightNumber,RecentDelays,Airline
0,LoNDon_paris,10045,"[23, 47]",KLM(!)
1,MAdrid_miLAN,10055,[],(12)
2,londON_StockhOlm,10065,"[24, 43, 87]",(British Airways. )
3,Budapest_PaRis,10075,[13],12. Air France
4,Brussels_londOn,10085,"[67, 32]","""Swiss Air"""


2. The From_To column would be better as two separate columns! Split each string on the underscore delimiter _ to give a new temporary DataFrame with the correct values. Assign the correct column names to this temporary DataFrame.

In [2]:
df2=pd.DataFrame(df)
df2[df2.columns[0].split("_")[0]] = df2['From_To'].apply(lambda x:x.split("_")[0])
df2[df2.columns[0].split("_")[1]] = df2['From_To'].apply(lambda x:x.split("_")[1])
new_columns = list(df2.columns[4:]) + list(df2.columns[1:4]) + list(df2.columns[0:1])
df2=pd.DataFrame(df2[new_columns])
df2

,From,To,FlightNumber,RecentDelays,Airline,From_To
0,LoNDon,paris,10045,"[23, 47]",KLM(!),LoNDon_paris
1,MAdrid,miLAN,10055,[],(12),MAdrid_miLAN
2,londON,StockhOlm,10065,"[24, 43, 87]",(British Airways. ),londON_StockhOlm
3,Budapest,PaRis,10075,[13],12. Air France,Budapest_PaRis
4,Brussels,londOn,10085,"[67, 32]","""Swiss Air""",Brussels_londOn


3. Notice how the capitalisation of the city names is all mixed up in this temporary DataFrame. Standardise the strings so that only the first letter is uppercase (e.g. "londON" should become "London".)

In [3]:
df2['From'] = df2['From'].apply(lambda x:x.lower().title())
df2['To'] = df2['To'].apply(lambda x:x.lower().title())
df2

,From,To,FlightNumber,RecentDelays,Airline,From_To
0,London,Paris,10045,"[23, 47]",KLM(!),LoNDon_paris
1,Madrid,Milan,10055,[],(12),MAdrid_miLAN
2,London,Stockholm,10065,"[24, 43, 87]",(British Airways. ),londON_StockhOlm
3,Budapest,Paris,10075,[13],12. Air France,Budapest_PaRis
4,Brussels,London,10085,"[67, 32]","""Swiss Air""",Brussels_londOn


4. Delete the From_To column from df and attach the temporary DataFrame from the previous questions.

In [57]:
df.drop(['From_To'],axis=1,inplace=True)
new_columns = list(df.columns[3:]) + list(df.columns[0:3])
df=pd.DataFrame(df[new_columns])
df['From'] = df2['From']
df['To'] = df2['To']
df

,FlightNumber,RecentDelays,Airline,From,To
0,10045,"[23, 47]",KLM(!),London,Paris
1,10055,[],(12),Madrid,Milan
2,10065,"[24, 43, 87]",(British Airways. ),London,Stockholm
3,10075,[13],12. Air France,Budapest,Paris
4,10085,"[67, 32]","""Swiss Air""",Brussels,London


5. In the RecentDelays column, the values have been entered into the DataFrame as a list. We would like each first value in its own column, each second value in its own column, and so on. If there isn't an Nth value, the value should be NaN.

Expand the Series of lists into a DataFrame named delays, rename the columns delay_1, delay_2, etc. and replace the unwanted RecentDelays column in df with delays.

In [58]:
delay_df=pd.DataFrame(df['RecentDelays'])
max_cols = 0
for val in delay_df['RecentDelays']:
    if len(val) > max_cols:
        max_cols = len(val)
print(max_cols)
for col in range(0,max_cols):
    delay_df['column_delay'+str(col+1)] = delay_df['RecentDelays'].apply(lambda x:np.nan if len(x) < (col+1) else x[col])

print(delay_df)
delay_df.drop(['RecentDelays'],axis=1,inplace=True)

df = pd.concat([df.loc[:,df.columns != 'RecentDelays'],delay_df],sort=False,axis=1)
df

3
   RecentDelays  column_delay1  column_delay2  column_delay3
0      [23, 47]           23.0           47.0            NaN
1            []            NaN            NaN            NaN
2  [24, 43, 87]           24.0           43.0           87.0
3          [13]           13.0            NaN            NaN
4      [67, 32]           67.0           32.0            NaN


,FlightNumber,Airline,From,To,column_delay1,column_delay2,column_delay3
0,10045,KLM(!),London,Paris,23.0,47.0,NaN
1,10055,(12),Madrid,Milan,NaN,NaN,NaN
2,10065,(British Airways. ),London,Stockholm,24.0,43.0,87.0
3,10075,12. Air France,Budapest,Paris,13.0,NaN,NaN
4,10085,"""Swiss Air""",Brussels,London,67.0,32.0,NaN
